En esta libreta implementaremos redes neuronales alimentadas hacia adelante con algunas funciones de activación.

Utilizamos la biblioteca NumPy para realizar cálculos eficientes con arreglos multidimensionales (vectores, matrices, etc).

In [ ]:
import numpy as np

Consideramos el problema que discutimos en clases, donde cada entrada $x$ consiste de dos componentes reales.

Estos son los datos de entrenamiento, vamos a considerar una entrada como un vector columna, por lo que las entradas de ejemplo serán una matriz de dos por cinco.

In [ ]:
inputs = np.array([[0, 2, 0, 2], [2, 0, 0, 2]])
outputs = np.array([+1, +1, -1, -1])

In [ ]:
print(f"  x     y")
print("----------")
for x, y in zip(inputs.T, outputs):
    print(x, f" {y:+g}")

El extractor de características agrega un valor unitario.

In [ ]:
def features(x):
    ones = np.ones(x.shape[1:])
    ones = ones.reshape((1,)+ones.shape)
    return np.vstack([ones, x])

La función `X` toma un índice `i` y regresa el `i`-ésimo vector columna de entrada

In [ ]:
def X(i):
    return inputs[:,i:i+1]

El problema se resuelve en dos etapas, primero se divide en dos subproblemas y luego las soluciones a estos son combinadas. Para ilustrar los cálculos, consideramos el primer ejemplo.

In [ ]:
fvec = features(X(0))
y = outputs[0]

En la primera etapa, consideramos la siguiente matriz de pesos, la cuál es multiplicada por el vector de características y posteriormente el resultado es evaluado por la función de paso.

In [ ]:
V = np.array([[-1, +1, -1],
              [-1, -1, +1]])

In [ ]:
def threshold(z):
    return (z >= 0) * 1

In [ ]:
hx = threshold(np.tensordot(V, fvec, axes=1))
hx

En la segunda etapa, consideramos combinar las soluciones de los subproblemas con la suma y calculando la función signo.

In [ ]:
w = np.array([1, 1])

In [ ]:
def sign(z):
    return (z > 0) * 2 - 1

In [ ]:
yest = sign(np.tensordot(w, hx, axes=1))
yest

Todo este proceso se puede realizar *de golpe* al procesar todos los datos de entrenamiento en lugar de un vector columna. De tal manera que el resultado es un vector con la predicción de cada ejemplo de entrenamiento.

In [ ]:
sign(np.tensordot(w, threshold(np.tensordot(V, features(inputs), axes=1)), axes=1))

In [ ]:
outputs

Implementamos la función `predict` que orquesta todas estas operaciones.

In [ ]:
def predict(V, w, input):
    fvec = features(input)
    Hvec = threshold(np.tensordot(V, fvec, axes=1))
    yvec = sign(np.tensordot(w, Hvec, axes=1))
    return yvec

In [ ]:
predict(V, w, X(0))

Visualicemos cómo este modelo clasifica puntos en el espacio.

In [ ]:
%matplotlib widget

import ipywidgets
import matplotlib.pyplot as plt
import matplotlib as mpl

plt.ioff();

In [ ]:
x1s = np.linspace(-3, 3, 100)
x2s = np.linspace(-3, 3, 100)
x1v, x2v = np.meshgrid(x1s, x2s)
yv = predict(V, w, np.stack([x1v, x2v]))

In [ ]:
fig, ax = plt.subplots()
h = ax.contourf(x1v, x2v, yv)
fig.colorbar(mpl.cm.ScalarMappable(mpl.colors.Normalize(-1, 1)), ax=ax)
fig.canvas.header_visible = False
display(fig.canvas)

¿Cómo afectan los parámetros de nuestra red neuronal a la clasificación? Hagamos nuestra visualización interactiva, usaremos la notación `V[i,j]` para referirnos a la componente en `V` del renglón `i` y columna `j`.

In [ ]:
def plot_classifier(predict):
    V11_init, V11_min, V11_max, V11_step = -1, -3, +3, 0.1
    V12_init, V12_min, V12_max, V12_step = +1, -3, +3, 0.1
    V13_init, V13_min, V13_max, V13_step = -1, -3, +3, 0.1
    V21_init, V21_min, V21_max, V21_step = -1, -3, +3, 0.1
    V22_init, V22_min, V22_max, V22_step = -1, -3, +3, 0.1
    V23_init, V23_min, V23_max, V23_step = +1, -3, +3, 0.1
    
    w1_init, w1_min, w1_max, w1_step = 1, -3, +3, 0.1
    w2_init, w2_min, w2_max, w2_step = 1, -3, +3, 0.1
    
    def weights(V11, V12, V13, V21, V22, V23, w1, w2):
        V = np.array([[V11, V12, V13],
                      [V21, V22, V23]])
        w = np.array([w1, w2])
        return V, w
    
    x1s = np.linspace(-3, 3, 100)
    x2s = np.linspace(-3, 3, 100)
    x1v, x2v = np.meshgrid(x1s, x2s)
    X = np.stack([x1v, x2v])
    
    fig, ax = plt.subplots()
    V_init, w_init = weights(V11_init, V12_init, V13_init, V21_init, V22_init, V23_init, w1_init, w2_init)
    yvs = predict(V_init, w_init, X)
    plot = ax.contourf(x1v, x2v, yvs)
    
    # fig.colorbar(mpl.cm.ScalarMappable(mpl.colors.Normalize(-1, 1)), ax=ax)
    cb = fig.colorbar(plot, ax=ax)

    def update_plot(V11, V12, V13, V21, V22, V23, w1, w2):
        nonlocal cb
        V, w = weights(V11, V12, V13, V21, V22, V23, w1, w2)
        yvs = predict(V, w, X)
        cb.remove()
        ax.clear()
        plot = ax.contourf(x1v, x2v, yvs)
        cb = fig.colorbar(plot, ax=ax)
        fig.canvas.draw()
        fig.canvas.flush_events()

    widget = ipywidgets.interactive(
        update_plot,
        V11 = ipywidgets.FloatSlider(
            orientation="horizontal",
            description="V[1,1]",
            value=V11_init,
            min=V11_min,
            max=V11_max,
            step=V11_step,
            layout=ipywidgets.Layout(width="90%"),
        ),
        V12 = ipywidgets.FloatSlider(
            orientation="horizontal",
            description="V[1,2]",
            value=V12_init,
            min=V12_min,
            max=V12_max,
            step=V12_step,
            layout=ipywidgets.Layout(width="90%"),
        ),
        V13 = ipywidgets.FloatSlider(
            orientation="horizontal",
            description="V[1,3]",
            value=V13_init,
            min=V13_min,
            max=V13_max,
            step=V13_step,
            layout=ipywidgets.Layout(width="90%"),
        ),
        V21 = ipywidgets.FloatSlider(
            orientation="horizontal",
            description="V[2,1]",
            value=V21_init,
            min=V21_min,
            max=V21_max,
            step=V21_step,
            layout=ipywidgets.Layout(width="90%"),
        ),
        V22 = ipywidgets.FloatSlider(
            orientation="horizontal",
            description="V[2,2]",
            value=V22_init,
            min=V22_min,
            max=V22_max,
            step=V22_step,
            layout=ipywidgets.Layout(width="90%"),
        ),
        V23 = ipywidgets.FloatSlider(
            orientation="horizontal",
            description="V[2,3]",
            value=V23_init,
            min=V23_min,
            max=V23_max,
            step=V23_step,
            layout=ipywidgets.Layout(width="90%"),
        ),
        w1 = ipywidgets.FloatSlider(
            orientation="horizontal",
            description="w[1]",
            value=w1_init,
            min=w1_min,
            max=w1_max,
            step=w1_step,
            layout=ipywidgets.Layout(width="90%"),
        ),
        w2 = ipywidgets.FloatSlider(
            orientation="horizontal",
            description="w[2]",
            value=w2_init,
            min=w2_min,
            max=w2_max,
            step=w2_step,
            layout=ipywidgets.Layout(width="90%"),
        ),
    )
    fig.canvas.header_visible = False
    display(widget)
    display(fig.canvas)
    return None

In [ ]:
plot_classifier(predict)

**Problema 1**

Implementa la función `make_predictor`, toma dos argumentos: `hidden_activation`, que es una función de activación para la capa oculta; `final_activation`, que es una función de activación para la capa final. Debe regresar un predictor con la estructura de la función `predict` de arriba.

In [ ]:
def make_predictor(hidden_activation, final_activation):
    def predict(V, w, input):
        raise NotImplementedError("Falta implementar make_predictor")
    return predict

Prueba tu implementación utilizando `plot_classifier` y un predictor construido con las funciones de activación `threshold` y `sign`. Deberías obtener una gráfica interactiva como la anterior.

In [ ]:
plot_classifier(make_predictor(threshold, sign))

**Problema 2**

Implementa la función de activación logística `logistic` y la función de activación rectificador `relu`. Observa que `logistic` regresa un valor entre $0$ y $1$, implementa una función constructora de sigmoides llamada `sigmoid` que regresa una función como `logistic` cuyos valores sean entre una cota inferior y una superior.

In [ ]:
def logistic(z):
    raise NotImplementedError("Falta implementar logistic")

In [ ]:
def relu(z):
    raise NotImplementedError("Falta implementar relu")

In [ ]:
def sigmoid(lower, upper):
    def activation(z):
        raise NotImplementedError("Falta implementar sigmoid")
    return activation

Prueba tus implementaciones utilizando `plot_classifier` y `make_predictor`.

Determina por qué se clasifican de la misma manera todos los puntos al usar `logistic` como activación oculta y `sign` como activación final. Puede resultarte útil no utilizar una función de activación en la capa final, puedes lograr esto modificando únicamente la siguiente celda.

In [ ]:
plot_classifier(make_predictor(logistic, sign))

**Problema 3**

Modela una red neuronal de $n$ capas utilizando los mecanismos de orientado a objetos de Python.
Investiga el algoritmo de retropropagación para aprender pesos utilizando la pérdida cuadrática y el descenso de gradiente estocástico.

Por el momento, verifica el correcto funcionamiento de tus redes neuronales generando conjuntos de datos sintéticos.